In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D 
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input 
import math 
import os


In [6]:
TRAIN_DATA_DIR = 'data/train/'
VALIDATION_DATA_DIR = 'data/val/'
TRAIN_SAMPLES = 500 
VALIDATION_SAMPLES = 500
NUM_CLASSES = 2 
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [7]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
#2017250045 정태환

In [8]:
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [9]:
def model_maker():
    base_model = MobileNetV2(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)
#2017250045 정태환

In [10]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])
model.fit_generator(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))
#2017250045 정태환

9420800/9406464 [==============================] - 1s 0us/step


C:\Users\JTH\AppData\Local\Temp\ipykernel_3332\2507997108.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
8/8 [==============================] - 34s 4s/step - loss: 0.8817 - acc: 0.6700 - val_loss: 0.2085 - val_acc: 0.9320
Epoch 2/10
8/8 [==============================] - 28s 4s/step - loss: 0.2591 - acc: 0.9080 - val_loss: 0.1039 - val_acc: 0.9700
Epoch 3/10
8/8 [==============================] - 28s 4s/step - loss: 0.1710 - acc: 0.9180 - val_loss: 0.0630 - val_acc: 0.9820
Epoch 4/10
8/8 [==============================] - 33s 4s/step - loss: 0.0987 - acc: 0.9660 - val_loss: 0.0686 - val_acc: 0.9720
Epoch 5/10
8/8 [==============================] - 33s 4s/step - loss: 0.0821 - acc: 0.9660 - val_loss: 0.0440 - val_acc: 0.9840
Epoch 6/10
8/8 [==============================] - 30s 4s/step - loss: 0.0887 - acc: 0.9680 - val_loss: 0.0425 - val_acc: 0.9840
Epoch 7/10
8/8 [==============================] - 30s 4s/step - loss: 0.0693 - acc: 0.9720 - val_loss: 0.0408 - val_acc: 0.9820
Epoch 8/10
8/8 [==============================] - 31s 4s/step - loss: 0.0787 - acc: 0.9740 - val_loss: 0

In [11]:
model.save('dogcat.h5')
#2017250045 정태환

In [12]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model('dogcat.h5')

img_path = 'data/val/dog/dog.250.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)

[[0.00246331 0.99753666]]
{'cat': 0, 'dog': 1}


In [13]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model('dogcat.h5')

img_path = 'data/val/cat/cat.250.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)

[[0.9987576  0.00124245]]
{'cat': 0, 'dog': 1}
